# pdata with QCoDeS example

This notebook shows how to use QCoDeS for instrument control but save data using procedural_data and use standard Python flow control.

The example also shows how to read the saved data back using analysis.dataview.

Changes to instrument settings between calls to add_points() are stored automatically.

The top-level Jupyter notebook (if any) will also be saved automatically in the data directory.

In [ ]:
%matplotlib widget

import matplotlib
import numpy as np
import matplotlib.pyplot as plt

import os
import sys
import logging
import time

In [ ]:
import qcodes as qc # <-- Assumes you've installed QCoDeS: https://qcodes.github.io/Qcodes/start/index.html

In [ ]:
from pdata.procedural_data import run_measurement

# Define data storage location

In [ ]:
data_root = r'example_data_root' # <-- path to your data root directory

# Create an artificial instrument driver, just for the sake of this example

In [ ]:
class FakeVNA(qc.instrument.base.Instrument):

  def __init__(self, name, **kwargs):
    super().__init__(name, **kwargs)
    self._power=0
    self.add_parameter('power', unit="dBm", get_cmd=lambda: self._power, set_cmd=self._setp)

  def _setp(self, val):
    """ Helper for keeping track of last set power value. """
    self._power=val

  def acquire_S21(self):
    """ Return some made up S21 (magnitude) data, with dependence on power. """
    time.sleep(0.5)
    freqs = np.linspace(5.9e9, 6.1e9, 41)
    lorenzian = lambda f,gamma,f0=6e9: 1/np.sqrt(np.pi) * (gamma / (gamma - 1j*(f-f0)))
    return freqs, lorenzian(freqs, 10e6 * 2**(self.power()/10.)) * (1 + 0.3*np.random.rand(len(freqs)))

  def ask(self, query):
    """ In some versions of QCoDeS, all instruments must support ask("*IDN?") """
    if query.strip().lower()=="*idn?": return "FakeVNA"
    else: assert False, "Not implemented."

# Init instrument and create a QCoDeS station containing one instrument

In [ ]:
vna = FakeVNA(name='vna')
station = qc.Station(vna)

# Measure

In [ ]:
# Define a function that gets the current instrument settings from QCoDeS (as a dict)
import qcodes.station
get_qcodes_instrument_snapshot = lambda s=qcodes.station.Station.default: s.snapshot(update=True)

In [ ]:
# Columns are specified as (<name>, <unit>), or just <name> if the quantity is dimensionless.
with run_measurement(get_qcodes_instrument_snapshot,
                     columns = [("frequency", "Hz"),
                                "S21"],
                     name='power-sweep', # <-- arbitrary str descriptive of measurement type
                     data_base_dir=data_root) as m:

  logging.warning('This test warning will (also) end up in log.txt within the data dir.')

  data_path = m.path()
  logging.warning(f'Data directory path = {m.path()}.')

  for p in [-30, -20, -10]:
    vna.power(p)  # <-- note that this new value gets automatically stored in the data
    freqs, s21 = vna.acquire_S21()
    m.add_points({'frequency': freqs, 'S21': s21})

# Read the data back using DataView

You should almost always **have analysis in a separate Jupyter notebook**. Here it's in the same one just to keep the demo in one place.

In [ ]:
from pdata.analysis.dataview import DataView, PDataSingle

In [ ]:
# Read the data from disk into a PDataSingle object
# and then feed that into a DataView object for analysis
#
# PDataSingle and Dataview are separate because you can
# concatenate multiple data dirs into one DataView by
# adding multiple PDataSingle's to the array below.
d = DataView([ PDataSingle(data_path), ])

In [ ]:
# Let's take a look at the HTML-formatted summary of d,
# before adding any virtual dimensions
d

In [ ]:
# Add a column to the data table based on a value from the settings snapshot.
# To figure out the path you need to specify in from_set.
# use the graphical helper dataexplorer.snapshot_explorer(d)
# See example at the end of this notebook.
d.add_virtual_dimension('VNA power', units="dBm",
                        from_set=('instruments', 'vna',
                                  'parameters', 'power', 'value'))

In [ ]:
# Let's take a look at the HTML-formatted summary of d again,
# note the new "VNA power" column, added with add_virtual_dimension() above.
d

## Looking at the data as arrays

In [ ]:
# Print some stored tabular data values:
d["frequency"][0::10]

In [ ]:
# Print some stored tabular data values:
d["S21"][0::10]

In [ ]:
# Above we defined VNA power as a "virtual column", so you can access that just the same as the real tabular data columns:
d["VNA power"][0::10]

In [ ]:
print('\nUnique powers in the data set: %s' % (np.unique(d["VNA power"])))

In [ ]:
# Divide the rows into contiguous ranges ("sweeps")
# based on on a parameter that stays constant during a single sweep
print('\nSweeps based on a per-sweep-fixed parameter: %s' % d.divide_into_sweeps('VNA power'))

In [ ]:
# Do the same based on on a parameter that increases or decreases monotonously during a single sweep
print('\nSweeps based on a per-sweep-swept parameter: %s' % d.divide_into_sweeps('frequency'))

In [ ]:
print('Instruments in the snapshot file:')
print(d.settings()[0][1]['instruments'].keys())

## Plot the sweeps manually

In [ ]:
fig, ax = plt.subplots()

for dd in d.sweeps('frequency'): # <-- split data rows into monotonously increasing/decreasing sweeps 
#for dd in d.sweeps('VNA power'): # <-- This works equally well, instead using a parameter that stays constant within a sweep.
  power = dd.single_valued_parameter('VNA power')
  ax.plot(dd['frequency'], np.abs(dd['S21']), label="%s dBm" % power)

ax.set(xlabel=f'f (Hz)', ylabel='S21')
ax.set_yscale('log')
ax.legend();

## Dataexplorer demo: Create a similar plots using dataexplorer to quickly examine sanity of data

In [ ]:
from pdata.analysis import dataexplorer

In [ ]:
# Create a graphical selector for choosing one or more data sets to plot (in the next cell)
sel = dataexplorer.data_selector(data_root)
display(sel)

In [ ]:
print(f"Plotting datasets: {sel.value}")
dataexplorer.basic_plot(data_root, sel.value, "frequency", "S21", ylog=True);

In [ ]:
# Same as above, but ignore phase by applying np.abs to y before plotting
dataexplorer.basic_plot(data_root, sel.value, "frequency", "S21", ylog=True, trace_processor=lambda x,y: (x,np.abs(y)));

### Slightly more complex example: Same plot as above, but with legend for VNA power added

In [ ]:
# Define a separate function that adds the virtual dimension(s)
# so that it can be passed as a "preprocessor" in basic_plot() below
def add_vdims(dd):
  """ Add virtual dimensions to DataView dd. """
  dd.add_virtual_dimension('VNA power', units="dBm",
                            from_set=('instruments', 'vna',
                                      'parameters', 'power', 'value'))
  return dd

In [ ]:
dataexplorer.basic_plot(data_root, sel.value,
                        "frequency", "S21",
                        slowcoordinate="VNA power",
                        ylog=True,
                        preprocessor=add_vdims);

### Snapshot explorer: Helper for setting up virtual dimensions

In [ ]:
dataexplorer.snapshot_explorer(d)

### Monitor a directory for new data (i.e. live plotting)

See separate notebook: pdata_with_qcodes_liveplot.ipynb